In [1]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}
url = "https://hvl.instructure.com/api/v1/accounts/1/users"

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict


# Finne gamle brukarar
Vi må ha ei rydderutine for å finne gamle eksterne brukarar. Dette er dei som er registret direkte i Canvas, og som ikkje har vore innlogga på "ei stund".

Først henter eg ut alle brukarar i Canvas:
```

In [2]:
dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
# Hent ut data (NB! dette vil variere frå endepunkt til endepunkt)
        dataliste = []
        for element in data:
            dataliste.append([element['id'], element['name'], element['sis_user_id'], element['created_at']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'sis_user_id', 'created_at'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei saman:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)


https://hvl.instructure.com/api/v1/accounts/1/users?page=2&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=3&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=4&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=5&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=6&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=7&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=8&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=9&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=10&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=11&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=12&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=13&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=14&per_page=100
https://hvl.instructure.com/api/v1/accounts/1/users?page=15

Så ser eg litt på kor mange det er og dei fem første:

In [3]:
alledata.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82835 entries, 0 to 82834
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           82835 non-null  int64 
 1   name         82835 non-null  object
 2   sis_user_id  82252 non-null  object
 3   created_at   82835 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [4]:
alledata.head()

,id,name,sis_user_id,created_at
0,1703,Terje Rudi (admin),hvl230817-p3-dp,2017-12-16T00:21:52+01:00
1,108712,Gunhild Skåsheim (student),HVL240919-P1-JEO,2024-09-19T22:43:10+02:00
2,96153,Terje Rudi (student),fs:203:265856,2023-08-17T11:48:55+02:00
3,54230,Head of International Relations-faculty of Soc...,hvl-200917-p5-jbru,2020-09-17T10:25:23+02:00
4,8600,Abeba Ababu,fs:203:421965,2018-03-27T16:34:42+02:00


## Dele opp etter type
Det er nokre eksterne brukarar som ikkje har `sis_user_id`, så eg plukker ut dei først i ei eiga liste (og tar dei vekk frå hovudlista):


In [5]:
utan_sis = alledata[alledata['sis_user_id'].isnull()]
med_sis = alledata[~alledata['sis_user_id'].isnull()]


In [6]:
med_sis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82252 entries, 0 to 82834
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           82252 non-null  int64 
 1   name         82252 non-null  object
 2   sis_user_id  82252 non-null  object
 3   created_at   82252 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.1+ MB


In [7]:
utan_sis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 583 entries, 157 to 82823
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           583 non-null    int64 
 1   name         583 non-null    object
 2   sis_user_id  0 non-null      object
 3   created_at   583 non-null    object
dtypes: int64(1), object(3)
memory usage: 22.8+ KB


Så kan eg finne dei i `med_sis` som *ikkje* starter med "fs:203" (og dei som starter med "fs:203"):

In [9]:
# eksterne_temp = med_sis[~(med_sis['sis_user_id'].str.lower.startswith("fs:203") | med_sis['sis_user_id'].str.startswith("FS:203") | med_sis['sis_user_id'].str.startswith("Fs:203"))]
eksterne_temp = med_sis[~(med_sis['sis_user_id'].str.contains("fs:203", case=False))]
interne = med_sis[(med_sis['sis_user_id'].str.contains("fs:203", case=False))]

Så er det igjen nokon av desse som kjem frå andre høgskular (via Trust account):

In [10]:
trust = eksterne_temp[eksterne_temp['sis_user_id'].str.startswith("fs:")] 
andre = eksterne_temp[~eksterne_temp['sis_user_id'].str.startswith("fs:")]

No har eg desse listene med data (antal i parentes):

- `utan_sis` er dei som ikkje har nokon SIS_ID (585)
- `med_sis` er dei som har SIS_ID. Desse er igjen delt inn i 
  - `trust` er dei som kjem frå andre institusjonar (som vi har Trust account-avtale med) (5409)
  - `interne` er HVL sine eigne (71161)
  - `andre` er alle andre (5354)

Kontroll: dette summerer seg til 82509, som er det eg henta frå Canvas.

Eg skriv alle desse til CSV-filar for seinare bruk:

In [11]:
utan_sis.to_csv("utan_sis.csv", index=False)
trust.to_csv("trust.csv", index=False)
interne.to_csv("interne.csv", index=False)
andre.to_csv("andre.csv", index=False)